# Book Recommendations.

In [69]:
import yaml
import sqlite3
from contextlib import contextmanager
from typing import List, Tuple, Dict
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize


class BooksRecommender:

    def __init__(self) -> None:
        
        # Model
        self.transformer = SentenceTransformer(
            "paraphrase-mpnet-base-v2" # English.
#             "paraphrase-multilingual-mpnet-base-v2" # Multilingual.
        )
        self.emb = None
        
        # Database.
        self.con = sqlite3.connect(":memory:")
        self.cur = self.con.cursor()
        self.cols = [
            "title",
            "description",
            "url",
            "image_url",
        ]
        self.cur.execute('''
        CREATE TABLE IF NOT EXISTS books (
              id integer not null primary key asc
            , title text
            , description text
            , url text
            , image_url text
        )
        ;
        ''')
        self.con.commit()
        
    def add_books(self,
                  values:List[Dict[str, str]]
                 ) -> None:
        values = [[d.get(c) for c in self.cols] for d in values]
        q = ",".join(["?" for c in self.cols])
        cols = ",".join([c for c in self.cols])
        self.cur.executemany(f'''
        INSERT INTO books ({cols}) values ({q})
        ''', values)
        self.con.commit()
        
    def update_by_id(self, _id:int, values:Dict[str, str]) -> None:
        k_v = ",".join([f"{k}=\"{v}\"" \
                        for k,v in values.items() if k in self.cols])
        self.cur.execute(f'''
        UPDATE books SET {k_v} WHERE id = ?
        ''', (_id,))
        self.con.commit()

    def get_by_title(self, title:str) -> List[Tuple[int, str, str]]:
        return [book for book in self.cur.execute('''
        SELECT * FROM books where title like ?
        ''', ("%"+title+"%",))]
    
    def get_by_id(self, _id:int) -> Tuple[int, str, str]:
        return [book for book in self.cur.execute('''
        SELECT * FROM books where id = ?
        ''', (_id,))]
    
    def apply_transformer(self) -> None:
        self.emb = normalize(
            self.transformer.encode(
            [t[0] for t in self.cur.execute(
            "select title || ' ' || description from books order by id asc;")]))
        
    def __call__(self, text:str, top_n:int=1) -> List[Tuple[str, str, str, str]]:
        query = self.transformer.encode([text]).T
        similarities = self.emb.dot(query)[:,0]
        ids = np.argpartition(similarities, len(similarities)-top_n)[-top_n:] + 1
        q = ",".join(["?" for _ in range(top_n)])
        return [t for t in self.cur.execute(
            f"select * from books where id in ({q})", ids.tolist())]

In [118]:
%%time
rec = BooksRecommender()
with open("books.yaml", "r") as f:
    rec.add_books(yaml.safe_load(f))
rec.apply_transformer()

CPU times: user 18.5 s, sys: 1.09 s, total: 19.6 s
Wall time: 18.6 s


In [130]:
%%time
rec("a story in medieval england", top_n=4)

CPU times: user 96.5 ms, sys: 6 ms, total: 102 ms
Wall time: 114 ms


[(10,
  'World Without End',
  'World Without End takes place in the same town of Kingsbridge, two centuries after the townspeople finished building the exquisite Gothic cathedral that was at the heart of The Pillars of the Earth. The cathedral and the priory are again at the center of a web of love and hate, greed and pride, ambition and revenge, but this sequel stands on its own. This time the men and women of an extraordinary cast of characters find themselves at a crossroads of new ideas—about medicine, commerce, architecture, and justice. In a world where proponents of the old ways fiercely battle those with progressive minds, the intrigue and tension quickly reach a boiling point against the devastating backdrop of the greatest natural disaster ever to strike the human race—the Black Death.',
  'https://www.goodreads.com/book/show/5201512-world-without-end',
  'https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1328202324l/5201512.jpg'),
 (12,
  'The Way of Kin